In [1]:
using PyPlot
using Laplacians

In [2]:
LAPLACIANS_NOAMG = true
LAPLACIANS_NOPLOT = true

true

In [19]:
data = readdlm("datasets/flip/flickr/Gsym_dense.csv",',')
data[:,1:2] = data[:,1:2] + 1
n = maximum(data[:,1:2])
m = size(data)
edlist = convert(Array{Int64,2}, data[:,1:2])
wts = convert(Array{Float64,1}, data[:,3])

957960-element Array{Float64,1}:
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 ⋮  
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0

In [20]:
a = sparse(edlist[:,1],edlist[:,2],wts,n,n)

7971×7971 SparseMatrixCSC{Float64,Int64} with 957960 stored entries:
  [188 ,    1]  =  1.0
  [230 ,    1]  =  1.0
  [426 ,    1]  =  1.0
  [480 ,    1]  =  1.0
  [738 ,    1]  =  1.0
  [817 ,    1]  =  1.0
  [1047,    1]  =  1.0
  [1338,    1]  =  1.0
  [1838,    1]  =  1.0
  [2408,    1]  =  1.0
  ⋮
  [6696, 7969]  =  1.0
  [6602, 7970]  =  1.0
  [6642, 7970]  =  1.0
  [6798, 7970]  =  1.0
  [7102, 7970]  =  1.0
  [7348, 7970]  =  1.0
  [5486, 7971]  =  1.0
  [6741, 7971]  =  1.0
  [7018, 7971]  =  1.0
  [7242, 7971]  =  1.0
  [7354, 7971]  =  1.0

In [21]:
maximum(edlist[:,2])

7971

In [30]:
JLfac=4.0
matrixConcConst=4.0
ep=4
f = approxCholLap(a,tol=1e-2);

n = size(a,1)
k = round(Int, JLfac*log(n)) # number of dims for JL

U = wtedEdgeVertexMat(a)
m = size(U,1)
R = randn(m,k)
UR = U'*R;

V = zeros(n,k)
for i in 1:k
V[:,i] = f(UR[:,i])
end

(ai,aj,av) = findnz(triu(a))
prs = zeros(av)
for h in 1:length(av)
  i = ai[h]
  j = aj[h]
  prs[h] = min(1,av[h]* (norm(V[i,:]-V[j,:])^2/k) * matrixConcConst *log(n)/ep^2)
end

ind = rand(size(prs)) .< prs

as = sparse(ai[ind],aj[ind],av[ind]./prs[ind],n,n)
as = as + as'

7971×7971 SparseMatrixCSC{Float64,Int64} with 35418 stored entries:
  [817 ,    1]  =  7.88617
  [3327,    1]  =  7.60532
  [1543,    2]  =  2.21282
  [1593,    2]  =  2.14013
  [1710,    2]  =  2.15506
  [3454,    2]  =  2.18527
  [605 ,    3]  =  32.3267
  [655 ,    3]  =  76.6224
  [963 ,    3]  =  12.3597
  [990 ,    3]  =  61.6492
  ⋮
  [5498, 7966]  =  1.37766
  [7421, 7967]  =  2.37487
  [6281, 7968]  =  3.10948
  [5465, 7969]  =  2.10211
  [5471, 7969]  =  2.11223
  [5981, 7969]  =  2.07701
  [6696, 7969]  =  2.05488
  [6602, 7970]  =  3.10492
  [6798, 7970]  =  2.98907
  [5486, 7971]  =  1.93967
  [7018, 7971]  =  1.8471

In [31]:
(ai,aj,av) = findnz(triu(as))

([30, 31, 6, 23, 67, 14, 12, 10, 5, 59  …  7421, 6281, 5465, 5471, 5981, 6696, 6602, 6798, 5486, 7018], [78, 78, 83, 90, 106, 118, 120, 132, 135, 159  …  7967, 7968, 7969, 7969, 7969, 7969, 7970, 7970, 7971, 7971], [2.99687, 3.0494, 190.592, 85.6267, 124.133, 58.6986, 22.1816, 4.39583, 8.35211, 12.24  …  2.37487, 3.10948, 2.10211, 2.11223, 2.07701, 2.05488, 3.10492, 2.98907, 1.93967, 1.8471])

In [32]:
ai = ai - 1
aj = aj - 1

17709-element Array{Int64,1}:
   77
   77
   82
   89
  105
  117
  119
  131
  134
  158
  158
  161
  167
    ⋮
 7964
 7965
 7966
 7967
 7968
 7968
 7968
 7968
 7969
 7969
 7970
 7970

In [33]:
minimum(aj),minimum(ai),maximum(aj),maximum(aj),

(77, 0, 7970, 7970)

In [34]:
fh = open("datasets/flip/flickr/Gsym_sp.csv","w")
  for i in 1:length(ai)
    write(fh, "$(ai[i]),$(aj[i]),$(av[i])\n")
    write(fh, "$(aj[i]),$(ai[i]),$(av[i])\n")
  end
close(fh)